In [10]:
# Install required libraries (run in your environment if needed)
# !pip install faker reportlab

from faker import Faker
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from datetime import datetime, timedelta
import random
import os

# Ensure output directory exists
output_dir = "out"
os.makedirs(output_dir, exist_ok=True)

# Initialize Faker
fake = Faker()

# Unified synthetic data generation for Wells Fargo
def generate_statement_data():
    """Generate synthetic data compatible with Wells Fargo style"""
    start = datetime(2025, 6, 1)
    end = datetime(2025, 6, 30)
    statement_period = f"{start.strftime('%B %d, %Y')} - {end.strftime('%B %d, %Y')}"
    account_type = random.choice(["Wells Fargo Everyday Checking", "Wells Fargo Business Checking"])
    account_holder = fake.name()
    account_holder_address_lines = [fake.street_address(), f"{fake.city()}, {fake.state_abbr()} {fake.zipcode()}"]
    customer_account_number = fake.bban()
    client_number = fake.ssn()
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%m/%d/%Y")
    
    # Generate transactions (deposits and withdrawals)
    deposit_descriptions = ["Direct Deposit", "Payroll Deposit", "Online Transfer", "Cash Deposit"]
    withdrawal_descriptions = ["ATM Withdrawal", "Grocery Purchase", "Utility Payment", "Online Purchase"]
    transactions = []
    beginning_balance = round(random.uniform(2000, 10000), 2)
    current_balance = beginning_balance
    
    # Generate deposits
    deposits = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(deposit_descriptions)}",
            "amount": round(random.uniform(300, 5000), 2)
        } for _ in range(random.randint(2, 5))
    ]
    
    # Generate withdrawals
    withdrawals = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(withdrawal_descriptions)}",
            "amount": -round(random.uniform(40, 1000), 2)
        } for _ in range(random.randint(3, 7))
    ]
    
    # Combine and sort transactions by date
    for item in deposits + withdrawals:
        transactions.append({
            "date": item["date"],
            "description": item["description"],
            "deposits_credits": f"${item['amount']:,.2f}" if item["amount"] >= 0 else "",
            "withdrawals_debits": f"${abs(item['amount']):,.2f}" if item["amount"] < 0 else "",
            "ending_balance": f"${current_balance + item['amount']:,.2f}"
        })
        current_balance += item["amount"]
    
    transactions.sort(key=lambda x: datetime.strptime(x["date"], "%m/%d/%Y"))
    
    # Calculate totals
    total_debit = sum(abs(item["amount"]) for item in withdrawals)
    total_credit = sum(item["amount"] for item in deposits)
    total_balance = current_balance
    
    return {
        "logo": os.path.join("sample_logos", "wellsfargo_logo.png"),
        "account_type": account_type,
        "statement_period": statement_period,
        "statement_date": end.strftime("%B %d, %Y"),
        "account_holder": account_holder,
        "account_holder_address_lines": account_holder_address_lines,
        "account_holder_address": ", ".join(account_holder_address_lines),
        "customer_account_number": customer_account_number,
        "client_number": client_number,
        "date_of_birth": date_of_birth,
        "customer_bank_name": "Wells Fargo Bank, N.A.",
        "transactions": transactions,
        "show_fee_waiver": random.choice([True, False]),
        "summary": {
            "beginning_balance": f"${beginning_balance:,.2f}",
            "deposits_total": f"${total_credit:,.2f}",
            "withdrawals_total": f"${total_debit:,.2f}",
            "ending_balance": f"${total_balance:,.2f}",
            "transactions_count": len(transactions)
        }
    }

# Generate shared data
ctx = generate_statement_data()

# ---------------------------------------------------------------------------------
# Wells Fargo Classic Template
# ---------------------------------------------------------------------------------

def create_wells_fargo_classic_statement(ctx):
    output_file = os.path.join(output_dir, f"wells_fargo_statement_classic_{ctx['customer_account_number'][-4:]}.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, leftMargin=0.5*inch, rightMargin=0.5*inch, topMargin=0.5*inch, bottomMargin=0.5*inch)
    elements = []

    # Styles based on HTML/CSS, reduced by 2 points
    styles = getSampleStyleSheet()
    statement_title_style = ParagraphStyle(
        name='StatementTitle', fontName='Helvetica-Bold', fontSize=20, leading=24, spaceAfter=4.5
    )
    subtitle_style = ParagraphStyle(
    name='Subtitle', fontName='Helvetica', fontSize=9, leading=12.6, spaceAfter=0
    )
    address_style = ParagraphStyle(
        name='Address', fontName='Helvetica', fontSize=10, leading=14, spaceAfter=0
    )
    help_box_style = ParagraphStyle(
        name='HelpBox', fontName='Helvetica', fontSize=9, leading=12.6, spaceAfter=3
    )
    help_box_bold_style = ParagraphStyle(
        name='HelpBoxBold', fontName='Helvetica-Bold', fontSize=10, leading=14, spaceAfter=3
    )
    intro_title_style = ParagraphStyle(
        name='IntroTitle', fontName='Helvetica-Bold', fontSize=14, leading=19.6, spaceAfter=4.5
    )
    intro_style = ParagraphStyle(
        name='Intro', fontName='Helvetica', fontSize=9.5, leading=13.3, spaceAfter=0
    )
    summary_title_style = ParagraphStyle(
        name='SummaryTitle', fontName='Helvetica-Bold', fontSize=13, leading=18.2, spaceAfter=3
    )
    summary_label_style = ParagraphStyle(
        name='SummaryLabel', fontName='Helvetica', fontSize=10, leading=14, spaceAfter=2.25
    )
    summary_num_style = ParagraphStyle(
        name='SummaryNum', fontName='Helvetica', fontSize=10, leading=14, spaceAfter=2.25, alignment=2
    )
    summary_bold_label_style = ParagraphStyle(
        name='SummaryBoldLabel', fontName='Helvetica-Bold', fontSize=10, leading=14, spaceAfter=2.25
    )
    routing_label_style = ParagraphStyle(
        name='RoutingLabel', fontName='Helvetica', fontSize=8.5, leading=11.9, spaceAfter=2.25
    )
    history_title_style = ParagraphStyle(
        name='HistoryTitle', fontName='Helvetica-Bold', fontSize=13, leading=18.2, spaceAfter=6
    )
    txn_head_style = ParagraphStyle(
        name='TxnHead', fontName='Helvetica-Bold', fontSize=10, leading=14
    )
    txn_row_style = ParagraphStyle(
        name='TxnRow', fontName='Helvetica', fontSize=10, leading=14
    )
    txn_num_style = ParagraphStyle(
        name='TxnNum', fontName='Helvetica', fontSize=10, leading=14, alignment=2
    )
    footer_style = ParagraphStyle(
        name='Footer', fontName='Helvetica', fontSize=8, leading=11.2, spaceAfter=3.75, alignment=4
    )

    PAGE_WIDTH, PAGE_HEIGHT = letter
    MARGIN = 0.5 * inch
    CONTENT_WIDTH = PAGE_WIDTH - 2 * MARGIN

    # Header
    header_content = []
    left_block = [
        Paragraph(ctx['account_type'], statement_title_style),
        Paragraph(f"Account number: {ctx['customer_account_number']} | {ctx['statement_period']}", subtitle_style)
    ]
    right_block = []
    logo_path = ctx['logo']
    if os.path.exists(logo_path):
        logo = Image(logo_path, width=50, height=50)
        right_block.append(logo)
    else:
        right_block.append(Paragraph("Logo not found: sample_logos\\wellsfargo_logo.png", subtitle_style))
    header_table = Table(
        [[left_block, right_block]],
        colWidths=[CONTENT_WIDTH * 0.65, CONTENT_WIDTH * 0.35],  # Adjusted from 0.6/0.4 to give more space to text
        rowHeights=[90]  # Increased from 80 to 90 to handle potential wrapping
    )
    header_table.setStyle(TableStyle([
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('ALIGN', (1,0), (1,0), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 27),
        ('BOTTOMPADDING', (0,0), (-1,-1), 13.5)
    ]))
    elements.append(header_table)

    # Address and Help Box
    addr_help_content = [
        [
            [Paragraph(line, address_style) for line in ctx['account_holder_address_lines']],  # Render each address line separately
            [
                Paragraph("Questions?", help_box_bold_style),
                Paragraph("Available by phone 24 hours a day, 7 days a week:", help_box_style),
                Paragraph("<b>1-800-CALL-WELLS</b> (1-800-225-5935)", help_box_style),
                Paragraph("TTY: 1-800-877-4833", help_box_style),
                Paragraph("En español: 1-877-337-7454", help_box_style),
                Spacer(1, 2.7),
                Paragraph("Online: wellsfargo.com", help_box_style),
                Spacer(1, 2.7),
                Paragraph("Write:", help_box_style),
                Paragraph("Wells Fargo Bank,", help_box_style),
                Paragraph("420 Montgomery Street", help_box_style),
                Paragraph("San Francisco, CA 94104", help_box_style)
            ]
        ]
    ]
    addr_help_table = Table(
        addr_help_content,
        colWidths=[CONTENT_WIDTH * 0.6, CONTENT_WIDTH * 0.35],
        style=[
            ('VALIGN', (0,0), (-1,-1), 'TOP'),
            ('LEFTPADDING', (0,0), (0,0), 0),
            ('RIGHTPADDING', (1,0), (1,0), 16.2),
            ('TOPPADDING', (0,0), (-1,-1), 12.15),
            ('BOTTOMPADDING', (0,0), (-1,-1), 12.15),
            ('LINEBEFORE', (1,0), (1,0), 1, colors.black)
        ]
    )
    elements.append(addr_help_table)

    # Intro Section
    intro_content = [
        Paragraph("Your Wells Fargo", intro_title_style),
        Paragraph(
            "It’s a great time to talk with a banker about how Wells Fargo’s accounts "
            "and services can help you stay competitive by saving you time and money. "
            "To find out how we can help, stop by any Wells Fargo location or call us at "
            "the number at the top of your statement.",
            intro_style
        )
    ]
    elements.append(Table([[intro_content]], colWidths=[CONTENT_WIDTH], style=[
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 16.2)
    ]))

    # Important Account Information
    info_content = [Paragraph("Important Account Information", intro_title_style)]
    if ctx['account_type'] == "Wells Fargo Everyday Checking":
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Everyday Checking accounts will increase to $12 unless you maintain a minimum daily balance of $500, have $500 in qualifying direct deposits, or maintain a linked Wells Fargo savings account with a balance of $300 or more.\n"
            "Starting June 30, 2025, Wells Fargo will introduce real-time transaction alerts for Everyday Checking accounts via the Wells Fargo Mobile app. Enable alerts at wellsfargo.com/alerts.\n"
            "Effective July 15, 2025, Wells Fargo will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for Everyday Checking accounts.\n"
            "For questions, visit wellsfargo.com or contact our Customer Service at 1-800-225-5935, available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Business Checking accounts will increase to $14 unless you maintain a minimum daily balance of $2,500 or have $1,000 in net purchases on a Wells Fargo Business Debit Card per month.\n"
            "Starting June 30, 2025, Wells Fargo will offer enhanced cash flow tools for Business Checking accounts via Wells Fargo Online Banking, including automated invoice tracking and payment scheduling.\n"
            "Effective July 15, 2025, Wells Fargo will reduce domestic wire transfer fees to $25 for Business Checking accounts, down from $30.\n"
            "For questions, visit wellsfargo.com or contact our Customer Service at 1-800-225-5935, available 24/7."
        )
    info_content.append(Paragraph(info_text, intro_style))
    elements.append(Table([[info_content]], colWidths=[CONTENT_WIDTH], style=[
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 16.2)
    ]))

    # Activity and Routing Summaries
    activity_data = [
        [Paragraph("Activity summary", summary_title_style), ""],
        [Paragraph("Beginning balance on", summary_label_style), Paragraph(ctx['summary']['beginning_balance'], summary_num_style)],
        [Paragraph("Deposits / Credits", summary_label_style), Paragraph(ctx['summary']['deposits_total'], summary_num_style)],
        [Paragraph("Withdrawals / Debits", summary_label_style), Paragraph(ctx['summary']['withdrawals_total'], summary_num_style)],
        [Paragraph("<b>Ending balance on</b>", summary_bold_label_style), Paragraph(f"<b>{ctx['summary']['ending_balance']}</b>", summary_num_style)]
    ]
    activity_table = Table(activity_data, colWidths=[CONTENT_WIDTH * 0.48 * 0.65, CONTENT_WIDTH * 0.48 * 0.35])
    activity_table.setStyle(TableStyle([
        ('LEFTPADDING', (0,0), (0,-1), 8.4375),  # 1.25em * 12pt * 0.9 -> 11.25pt * 0.75
        ('RIGHTPADDING', (1,0), (1,-1), 6),  # Increased from 0 to 6pt to push numbers left of the vertical line
        ('TOPPADDING', (0,0), (-1,-1), 2.7),
        ('BOTTOMPADDING', (0,0), (-1,-1), 2.7)
    ]))

    routing_data = [
        [Paragraph("", summary_title_style)],
        [Paragraph(f"Account number: {ctx['customer_account_number']}", routing_label_style)],
        [Paragraph(ctx['account_holder'], routing_label_style)],
        [Paragraph("For Direct Deposit and Automatic Payments use Routing Number (RTN): 053000219", routing_label_style)],
        [Paragraph("For Wire Transfer use Routing Number (RTN): 121000248", routing_label_style)]
    ]
    routing_table = Table(routing_data, colWidths=[CONTENT_WIDTH * 0.48])
    routing_table.setStyle(TableStyle([
        ('LEFTPADDING', (0,0), (-1,-1), 6),  # Increased from 0 to 6pt to push text right of the vertical line
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 2.7),
        ('BOTTOMPADDING', (0,0), (-1,-1), 2.7)
    ]))

    two_col_table = Table(
        [[activity_table, routing_table]],
        colWidths=[CONTENT_WIDTH * 0.48, CONTENT_WIDTH * 0.48],
        style=[
            ('LEFTPADDING', (0,0), (0,0), 0),
            ('RIGHTPADDING', (0,0), (0,0), 22.2),  # Increased from 16.2pt to 22.2pt (16.2 + 6)
            ('LEFTPADDING', (1,0), (1,0), 6),  # Increased from 0 to 6pt for extra space
            ('RIGHTPADDING', (1,0), (1,0), 0),
            ('TOPPADDING', (0,0), (-1,-1), 0),
            ('BOTTOMPADDING', (0,0), (-1,-1), 0),
            ('LINEBEFORE', (1,0), (1,0), 1, colors.black)
        ]
    )
    elements.append(Table([[two_col_table]], colWidths=[CONTENT_WIDTH], style=[
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0)
    ]))
    elements.append(Spacer(1, 8.1))
    elements.append(Table([[None]], colWidths=[CONTENT_WIDTH], rowHeights=[0.675], style=[
        ('BACKGROUND', (0,0), (-1,-1), colors.black)
    ]))
    elements.append(Spacer(1, 8.1))

    # Transaction History
    transactions_data = [
        [
            Paragraph("Date", txn_head_style),
            Paragraph("Description", txn_head_style),
            Paragraph("Deposits / Credits", txn_head_style),
            Paragraph("Withdrawals / Debits", txn_head_style),
            Paragraph("Ending daily balance", txn_head_style)
        ],
        [
            Paragraph("", txn_row_style),
            Paragraph("Opening balance", txn_row_style),
            Paragraph("", txn_num_style),
            Paragraph("", txn_num_style),
            Paragraph(ctx['summary']['beginning_balance'], txn_num_style)
        ]
    ]
    for transaction in ctx['transactions']:
        transactions_data.append([
            Paragraph(transaction["date"], txn_row_style),
            Paragraph(transaction["description"], txn_row_style),
            Paragraph(transaction["deposits_credits"], txn_num_style),
            Paragraph(transaction["withdrawals_debits"], txn_num_style),
            Paragraph(transaction["ending_balance"], txn_num_style)
        ])
    transactions_data.append([
        Paragraph("", txn_row_style),
        Paragraph("Total", txn_row_style),
        Paragraph(ctx['summary']['deposits_total'], txn_num_style),
        Paragraph(ctx['summary']['withdrawals_total'], txn_num_style),
        Paragraph(ctx['summary']['ending_balance'], txn_num_style)
    ])
    transactions_table = Table(
        transactions_data,
        colWidths=[CONTENT_WIDTH * 0.12, CONTENT_WIDTH * 0.40, CONTENT_WIDTH * 0.16, CONTENT_WIDTH * 0.16, CONTENT_WIDTH * 0.16]
    )
    transactions_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#d9d9d9")),
        ('BACKGROUND', (0,1), (-1,1), colors.HexColor("#d9d9d9")),
        ('BACKGROUND', (0,-1), (-1,-1), colors.HexColor("#d9d9d9")),
        ('LINEBELOW', (0,0), (-1,-2), 1, colors.HexColor("#666")),
        ('ALIGN', (2,0), (-1,-1), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 4.05),
        ('RIGHTPADDING', (0,0), (-1,-1), 4.05),
        ('TOPPADDING', (0,0), (-1,-1), 2.7),
        ('BOTTOMPADDING', (0,0), (-1,-1), 2.7)
    ]))
    elements.append(Table([[Paragraph("Transaction history", history_title_style)]], colWidths=[CONTENT_WIDTH], style=[
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 8.1),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0)
    ]))
    elements.append(transactions_table)

    # Footer
    footer_content = [
        Paragraph("Disclosures", footer_style),
        Paragraph(
            "All account transactions are subject to the Wells Fargo Deposit Account Agreement, available at wellsfargo.com. "
            "Interest rates and Annual Percentage Yields (APYs) may change without notice. For details on overdraft policies and fees, "
            "visit wellsfargo.com/overdraft or call 1-800-225-5935.",
            footer_style
        ),
        Paragraph("© 2025 Wells Fargo Bank, N.A. All rights reserved. Member FDIC.", footer_style)
    ]
    elements.append(Table([[footer_content]], colWidths=[CONTENT_WIDTH], style=[
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 24.3),
        ('BOTTOMPADDING', (0,0), (-1,-1), 32.4)
    ]))

    # Build PDF
    doc.build(elements)
    print(f"PDF generated: {output_file}")

# ---------------------------------------------------------------------------------
# Main Execution
# ---------------------------------------------------------------------------------

if __name__ == "__main__":
    create_wells_fargo_classic_statement(ctx)

PDF generated: out\wells_fargo_statement_classic_4193.pdf
